In [67]:
from pyspark.sql import SparkSession
import pyspark
import pyspark.sql.functions as f
from pyspark.sql.dataframe import DataFrame
import os

In [2]:
spark = SparkSession.builder.master('local[*]') \
    .appName('visualizacao') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/09/12 19:57:15 WARN Utils: Your hostname, DESKTOP-N81ME1R resolves to a loopback address: 127.0.1.1; using 172.24.108.240 instead (on interface eth0)
23/09/12 19:57:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/12 19:57:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [60]:
df = spark.read.json('../data/datalake/bronze/extract_date=*/*.json')
df.printSchema()
df = df.na.drop()
type(df)

root
 |-- data_extracao: string (nullable = true)
 |-- hr: string (nullable = true)
 |-- l: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- c: string (nullable = true)
 |    |    |-- cl: long (nullable = true)
 |    |    |-- lt0: string (nullable = true)
 |    |    |-- lt1: string (nullable = true)
 |    |    |-- qv: long (nullable = true)
 |    |    |-- sl: long (nullable = true)
 |    |    |-- vs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |    |-- is: string (nullable = true)
 |    |    |    |    |-- p: long (nullable = true)
 |    |    |    |    |-- px: double (nullable = true)
 |    |    |    |    |-- py: double (nullable = true)
 |    |    |    |    |-- sv: string (nullable = true)
 |    |    |    |    |-- ta: string (nullable = true)



pyspark.sql.dataframe.DataFrame

In [4]:
df_p = df.select(df.data_extracao, df.hr, f.explode(df.l).alias('lista'))
df_p.show(5, truncate=False)
df_p.printSchema()


+----------------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|data_extracao   |hr   |lista                                                                                                                                                                                                                                                                                       

In [5]:
df_p2 = df_p.select(df_p.data_extracao, df_p.lista, df_p.hr)
df_p2.printSchema()


root
 |-- data_extracao: string (nullable = true)
 |-- lista: struct (nullable = true)
 |    |-- c: string (nullable = true)
 |    |-- cl: long (nullable = true)
 |    |-- lt0: string (nullable = true)
 |    |-- lt1: string (nullable = true)
 |    |-- qv: long (nullable = true)
 |    |-- sl: long (nullable = true)
 |    |-- vs: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |-- is: string (nullable = true)
 |    |    |    |-- p: long (nullable = true)
 |    |    |    |-- px: double (nullable = true)
 |    |    |    |-- py: double (nullable = true)
 |    |    |    |-- sv: string (nullable = true)
 |    |    |    |-- ta: string (nullable = true)
 |-- hr: string (nullable = true)



In [6]:
df_dados_completos_linha = spark.read.options(header='True') \
.csv( '../data/datalake/bronze/relacao_empresas_linha/dados_linhas_completo.csv')
df_dados_completos_linha.show()

23/09/12 19:57:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/dados_linhas_completo.csv
+---+---+-------+-----------+--------------------+--------------------+
|_c0|_c1|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|
+---+---+-------+-----------+--------------------+--------------------+
|  0|  0|975A-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO SANTA BRIG...|
|  1|  1|9784-10|          1|CONSÓRCIO TRANSNO...|SPENCER TRANSPORT...|
|  2|  2|9785-10|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|  3|  3|9785-51|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|  4|  4|9789-10|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|  5|  5|978A-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO GATO PRETO...|

In [7]:
# df_amostra_um = df_p2.select(df_p2.data_extracao,df_p2.lista.c ,df_p2.lista.vs.p,df_p2.lista.vs.py, df_p2.lista.vs.px)
# Método para obter o dados dos ônibus
df_amostra_um = df_p2.select(
    df_p2.data_extracao,
    df_p2.hr,
    df_p2.lista.c.alias('LETREIRO_COMPLETO'), 
    df_p2.lista.sl.alias('SENTIDO_OPERACAO'), 
    df_p2.lista.cl.alias('CODIGO_IDENTIFICADOR_LINHA') , 
    f.explode(df_p2.lista.vs).alias('expansao')) \
    .select(f.col('data_extracao').alias('DATA_EXTRACAO'),
            f.col('hr').alias('HORA_API'),
            'LETREIRO_COMPLETO',
            'SENTIDO_OPERACAO',
            'CODIGO_IDENTIFICADOR_LINHA',
            f.col('expansao.p').alias('PREFIXO_ONIBUS'),
            f.col('expansao.ta').alias('DATA_HORA_CAPTURA_LOCALIZACAO'),
            f.col('expansao.py').alias('LATITUDE') ,
            f.col('expansao.px').alias('LONGITUDE'),
            
            ) \
    .sort(df_p2.data_extracao.desc(),df_p2.lista.c.asc() ) \
    
df_amostra_um.show(truncate=False)


+----------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+
|DATA_EXTRACAO   |HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|LATITUDE           |LONGITUDE          |
+----------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+
|2023-09-09 22:40|22:40   |1012-10          |2               |35254                     |16398         |2023-09-10T01:39:33Z         |-23.437934749999997|-46.801423125      |
|2023-09-09 22:40|22:40   |1012-10          |1               |2486                      |16479         |2023-09-10T01:39:55Z         |-23.4325754375     |-46.786764125000005|
|2023-09-09 22:40|22:40   |1012-10          |1               |2486                      |16475         |2023-09-10T01:39:56Z 

In [8]:
colunas_drop = ('_c0', '_c1', 'LINHA')

In [9]:
df_operacao_desagrupada =  df_dados_completos_linha.join(
    df_amostra_um,
    df_amostra_um.LETREIRO_COMPLETO == df_dados_completos_linha.LINHA , 
    'inner'
)
df_operacao_desagrupada = df_operacao_desagrupada.drop(*colunas_drop)
df_operacao_desagrupada = df_operacao_desagrupada.withColumn("DATA_EXTRACAO", f.to_date("DATA_EXTRACAO"))  
df_operacao_desagrupada = df_operacao_desagrupada.withColumn("DATA_EXTRACAO_API", f.to_date("DATA_EXTRACAO"))  
df_operacao_desagrupada.show(truncate=False)

+-----------+-------------------------------------+-------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+--------------+-----------------+
|CODIGO_AREA|CONSOCIO                             |EMPRESA                              |DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|LATITUDE           |LONGITUDE     |DATA_EXTRACAO_API|
+-----------+-------------------------------------+-------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+--------------+-----------------+
|6          |TRANSWOLFF TRANSPORTES E TURISMO LTDA|TRANSWOLFF TRANSPORTES E TURISMO LTDA|2023-09-09   |11:36   |6069-10          |1               |1154                      |66525  

In [11]:
df_operacao_desagrupada.write\
    .partitionBy('DATA_EXTRACAO')\
    .mode('overwrite')\
    .json('../data/datalake/prata/operacao_desagrupada/teste')

In [12]:
# Método de obter operação
df_operacao =df.select(df.hr, df.data_extracao, f.explode('l').alias('DADOS_LINHA')) \
    .withColumnRenamed('hr', 'HORA_API') \
    .select('HORA_API',
            'DATA_EXTRACAO',
            f.col('DADOS_LINHA.c').alias('LETREIRO_COMPLETO'),
            f.col('DADOS_LINHA.cl').alias('CODIGO_IDENTIFICADOR'), 
            f.col('DADOS_LINHA.lt0').alias('LETREIRO_ORIGEM'),
            f.col('DADOS_LINHA.lt1').alias('LETREIRO_DESTINO'), 
            f.col('DADOS_LINHA.qv').alias('QTDE_VEICULOS_OPERACAO'),
              )
#     .where('c == "7005-51"') \

df_operacao.show(truncate=False)

+--------+----------------+-----------------+--------------------+---------------------+-------------------+----------------------+
|HORA_API|DATA_EXTRACAO   |LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|LETREIRO_ORIGEM      |LETREIRO_DESTINO   |QTDE_VEICULOS_OPERACAO|
+--------+----------------+-----------------+--------------------+---------------------+-------------------+----------------------+
|11:36   |2023-09-09 11:36|6069-10          |1154                |TERM. STO. AMARO     |JD. SÃO BERNARDO   |4                     |
|11:36   |2023-09-09 11:36|6L10-41          |34773               |TERM. VARGINHA       |MESSIÂNICA         |4                     |
|11:36   |2023-09-09 11:36|2734-10          |34528               |METRÔ ITAQUERA       |JD. CAMPOS         |3                     |
|11:36   |2023-09-09 11:36|701A-10          |33457               |METRÔ VL. MADALENA   |PQ. EDU CHAVES     |8                     |
|11:36   |2023-09-09 11:36|273R-10          |32994               |METRÔ ARTU

In [13]:
df_dados_completos_linha.show()

23/09/10 22:39:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/dados_linhas_completo.csv
+---+---+-------+-----------+--------------------+--------------------+
|_c0|_c1|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|
+---+---+-------+-----------+--------------------+--------------------+
|  0|  0|975A-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO SANTA BRIG...|
|  1|  1|9784-10|          1|CONSÓRCIO TRANSNO...|SPENCER TRANSPORT...|
|  2|  2|9785-10|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|  3|  3|9785-51|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|  4|  4|9789-10|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|  5|  5|978A-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO GATO PRETO...|

In [14]:
df_dados_completos_operacao = df_dados_completos_linha.join(df_operacao, df_dados_completos_linha.LINHA == df_operacao.LETREIRO_COMPLETO, 'inner') 
df_dados_completos_operacao = df_dados_completos_operacao.withColumn('DATA_EXTRACAO', f.to_date('DATA_EXTRACAO'))
df_dados_completos_operacao.show(truncate=False)

23/09/10 22:39:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
 Schema: _c0, _c1, LINHA, CODIGO_AREA, CONSOCIO, EMPRESA
Expected: _c0 but found: 
CSV file: file:///home/rodrigo/projetos/monitoramento_sptrans/data/datalake/bronze/relacao_empresas_linha/dados_linhas_completo.csv
+----+---+-------+-----------+---------------------------------------+---------------------------------------+--------+-------------+-----------------+--------------------+---------------------+-------------------+----------------------+
|_c0 |_c1|LINHA  |CODIGO_AREA|CONSOCIO                               |EMPRESA                                |HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|LETREIRO_ORIGEM      |LETREIRO_DESTINO   |QTDE_VEICULOS_OPERACAO|
+----+---+-------+-----------+---------------------------------------+---------------------------------------+--------+-------------+-----------------+--------------------

In [16]:
colunas = ('_c0', '_c1')
df_dados_completos_operacao = df_dados_completos_operacao.drop(*colunas)
df_dados_completos_operacao.show()

+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+--------------------+-------------------+----------------------+
|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|     LETREIRO_ORIGEM|   LETREIRO_DESTINO|QTDE_VEICULOS_OPERACAO|
+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+--------------------+-------------------+----------------------+
|6069-10|          6|TRANSWOLFF TRANSP...|TRANSWOLFF TRANSP...|   11:36|   2023-09-09|          6069-10|                1154|    TERM. STO. AMARO|   JD. SÃO BERNARDO|                     4|
|6L10-41|          6|TRANSWOLFF TRANSP...|TRANSWOLFF TRANSP...|   11:36|   2023-09-09|          6L10-41|               34773|      TERM. VARGINHA|         MESSIÂNICA|                     4|
|2734-10|          3|UPBUS QUALIDADE E...|UPBUS QU

In [17]:
df_dados_completos_operacao.write\
    .partitionBy('DATA_EXTRACAO')\
    .mode('overwrite')\
    .json('../data/datalake/prata/operacao_agrupada/teste')

In [18]:
def export_json(df, dest):
    df.coalesce(1).write.mode("overwrite").json(dest)


In [19]:
export_json(df_dados_completos_operacao,'data/datalake/prata/operacao_agrupada/teste' )

# passando para funções

In [47]:
def dataset_load_bronze():
    df = spark.read.json('../data/datalake/bronze/extract_date=*/*.json')
    df = df.na.drop()
    return df

In [48]:
dataset_load_bronze().show()

+----------------+-----+--------------------+
|   data_extracao|   hr|                   l|
+----------------+-----+--------------------+
|2023-09-09 11:36|11:36|[{6069-10, 1154, ...|
|2023-09-09 11:40|11:40|[{6069-10, 1154, ...|
|2023-09-09 11:44|11:44|[{6069-10, 33922,...|
|2023-09-09 11:48|11:48|[{6069-10, 33922,...|
|2023-09-09 11:52|11:52|[{6069-10, 33922,...|
|2023-09-09 11:56|11:56|[{6069-10, 33922,...|
|2023-09-09 12:00|12:00|[{6069-10, 33922,...|
|2023-09-09 12:04|12:04|[{6069-10, 33922,...|
|2023-09-09 12:08|12:08|[{6069-10, 33922,...|
|2023-09-09 12:12|12:12|[{6069-10, 33922,...|
|2023-09-09 12:16|12:16|[{6069-10, 33922,...|
|2023-09-09 12:20|12:20|[{6069-10, 33922,...|
|2023-09-09 12:24|12:24|[{6069-10, 33922,...|
|2023-09-09 12:28|12:28|[{6069-10, 33922,...|
|2023-09-09 12:32|12:32|[{6069-10, 33922,...|
|2023-09-09 12:36|12:36|[{6069-10, 33922,...|
|2023-09-09 12:40|12:40|[{6069-10, 33922,...|
|2023-09-09 12:44|12:44|[{6069-10, 33922,...|
|2023-09-09 12:48|12:48|[{6069-10,

In [40]:
dataset_load_bronze().printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- hr: string (nullable = true)
 |-- lista: struct (nullable = true)
 |    |-- c: string (nullable = true)
 |    |-- cl: long (nullable = true)
 |    |-- lt0: string (nullable = true)
 |    |-- lt1: string (nullable = true)
 |    |-- qv: long (nullable = true)
 |    |-- sl: long (nullable = true)
 |    |-- vs: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |-- is: string (nullable = true)
 |    |    |    |-- p: long (nullable = true)
 |    |    |    |-- px: double (nullable = true)
 |    |    |    |-- py: double (nullable = true)
 |    |    |    |-- sv: string (nullable = true)
 |    |    |    |-- ta: string (nullable = true)



# Função agrupada

In [70]:
def operacao_agrupada(df: DataFrame)  -> DataFrame:
    df_operacao =df.select(df.hr, df.data_extracao, f.explode('l').alias('DADOS_LINHA')) \
    .withColumnRenamed('hr', 'HORA_API') \
    .select('HORA_API',
            'DATA_EXTRACAO',
            f.col('DADOS_LINHA.c').alias('LETREIRO_COMPLETO'),
            f.col('DADOS_LINHA.cl').alias('CODIGO_IDENTIFICADOR'), 
            f.col('DADOS_LINHA.lt0').alias('LETREIRO_ORIGEM'),
            f.col('DADOS_LINHA.lt1').alias('LETREIRO_DESTINO'), 
            f.col('DADOS_LINHA.qv').alias('QTDE_VEICULOS_OPERACAO'),
              )
    return df_operacao

In [71]:
dataset_bronze = dataset_load_bronze()

operacao_agrupada(dataset_bronze).show()

+--------+----------------+-----------------+--------------------+--------------------+-------------------+----------------------+
|HORA_API|   DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|     LETREIRO_ORIGEM|   LETREIRO_DESTINO|QTDE_VEICULOS_OPERACAO|
+--------+----------------+-----------------+--------------------+--------------------+-------------------+----------------------+
|   11:36|2023-09-09 11:36|          6069-10|                1154|    TERM. STO. AMARO|   JD. SÃO BERNARDO|                     4|
|   11:36|2023-09-09 11:36|          6L10-41|               34773|      TERM. VARGINHA|         MESSIÂNICA|                     4|
|   11:36|2023-09-09 11:36|          2734-10|               34528|      METRÔ ITAQUERA|         JD. CAMPOS|                     3|
|   11:36|2023-09-09 11:36|          701A-10|               33457|  METRÔ VL. MADALENA|     PQ. EDU CHAVES|                     8|
|   11:36|2023-09-09 11:36|          273R-10|               32994|   METRÔ ARTUR AL

# Operação desagrupadas

In [72]:
def operacao_desagrupada(df: DataFrame) -> DataFrame:
    df_p = df.select(df.data_extracao, df.hr, f.explode(df.l).alias('lista'))
    df_p = df_p.select(df_p.data_extracao, df_p.lista, df_p.hr)
    
    df_amostra_um = df_p.select(
    df_p.data_extracao,
    df_p.hr,
    df_p.lista.c.alias('LETREIRO_COMPLETO'), 
    df_p.lista.sl.alias('SENTIDO_OPERACAO'), 
    df_p.lista.cl.alias('CODIGO_IDENTIFICADOR_LINHA') , 
    f.explode(df_p.lista.vs).alias('expansao')) \
    .select(f.col('data_extracao').alias('DATA_EXTRACAO'),
            f.col('hr').alias('HORA_API'),
            'LETREIRO_COMPLETO',
            'SENTIDO_OPERACAO',
            'CODIGO_IDENTIFICADOR_LINHA',
            f.col('expansao.p').alias('PREFIXO_ONIBUS'),
            f.col('expansao.ta').alias('DATA_HORA_CAPTURA_LOCALIZACAO'),
            f.col('expansao.py').alias('LATITUDE') ,
            f.col('expansao.px').alias('LONGITUDE'),
            
            ) \
    .sort(df_p.data_extracao.desc(),df_p.lista.c.asc() ) 

    return df_amostra_um
    

In [74]:
operacao_desagrupada(dataset_bronze).show()

+----------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+
|   DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|           LATITUDE|          LONGITUDE|
+----------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+
|2023-09-09 22:40|   22:40|          1012-10|               2|                     35254|         16398|         2023-09-10T01:39:33Z|-23.437934749999997|      -46.801423125|
|2023-09-09 22:40|   22:40|          1012-10|               1|                      2486|         16479|         2023-09-10T01:39:55Z|     -23.4325754375|-46.786764125000005|
|2023-09-09 22:40|   22:40|          1012-10|               1|                      2486|         16475|         2023-09-10T0

In [91]:
def dados_completos_onibus():
    df_dados_completos_linha = spark.read.options(header='True') \
    .csv( '../data/datalake/bronze/relacao_empresas_linha/dados_linhas_completo.csv') \
    .drop(*('_c0', '_c1'))
    return df_dados_completos_linha

In [100]:
def juncao_dataframe(df_um: DataFrame, df_dois: DataFrame, coluna_um: str, coluna_dois: str, tipo_juncao: str = 'inner') -> DataFrame:
    df_dados_completos_operacao = df_um.join(df_dois, f.col(coluna_um) == f.col(coluna_dois), tipo_juncao)
    df_dados_completos_operacao = df_dados_completos_operacao.withColumn('DATA_EXTRACAO', f.to_date('DATA_EXTRACAO'))
    colunas = ('_c0', '_c1') 
    df_dados_completos_operacao = df_dados_completos_operacao.drop(*colunas)
    return df_dados_completos_operacao


In [99]:
juncao_dataframe(dados_completos_onibus(), operacao_desagrupada(dataset_bronze), 'LINHA', 'LETREIRO_COMPLETO').show()

+-------+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+--------------+
|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|           LATITUDE|     LONGITUDE|
+-------+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+--------------+
|6069-10|          6|TRANSWOLFF TRANSP...|TRANSWOLFF TRANSP...|   2023-09-09|   11:36|          6069-10|               1|                      1154|         66525|         2023-09-09T14:35:55Z|      -23.653890625|  -46.71154075|
|6069-10|          6|TRANSWOLFF TRANSP...|TRANSWOLFF TRANSP...|   2023-09-09|   11:3

In [92]:
dados_completos_onibus().show()


+-------+-----------+--------------------+--------------------+
|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|
+-------+-----------+--------------------+--------------------+
|975A-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO SANTA BRIG...|
|9784-10|          1|CONSÓRCIO TRANSNO...|SPENCER TRANSPORT...|
|9785-10|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|9785-51|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|9789-10|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|978A-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO GATO PRETO...|
|978J-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO SANTA BRIG...|
|978L-10|          1|CONSÓRCIO BANDEIR...|VIAÇÃO SANTA BRIG...|
|978T-10|          1|CONSÓRCIO TRANSNO...|NORTE BUSS TRANSP...|
|N101-11|          1|CONSÓRCIO BANDEIR...|VIAÇÃO SANTA BRIG...|
|N102-11|          1|CONSÓRCIO BANDEIR...|VIAÇÃO SANTA BRIG...|
|N103-11|          1|CONSÓRCIO BANDEIR...|VIAÇÃO SANTA BRIG...|
|N104-11|          1|CONSÓRCIO BANDEIR..